# creating dataset using github repo

Credits to: [alessandro1802](https://github.com/alessandro1802) for the repo: [page-recommender-system](https://github.com/alessandro1802/page-recommender-system), <br>
Where I found a lot of [Wikipedia](https://en.wikipedia.org/wiki/Wiki) pages scraped into txt files.

In [1]:
import requests
from tqdm.auto import tqdm
import string
import re

In [2]:
# provide values for the target github location containing data files
user = '<provide value>'
repo_name = '<provide value>'
path_to_file = '<provide value>'

In [3]:
# entering a target github repo containing scraped wikipedia pages
# credits: https://github.com/alessandro1802/page-recommender-system
user = 'alessandro1802'
repo_name = 'page-recommender-system'
path_to_file = 'database'

In [4]:
# get list of files in dataset
url = f'''https://api.github.com/repos/{user}/{repo_name}/contents/{path_to_file}'''
print(url)
content = {}
res = requests.get(url)
if res.status_code == requests.codes.ok:
    content = res.json()
    print('Content fetched successfully!')
else:
    print('Content was not found!')

https://api.github.com/repos/alessandro1802/page-recommender-system/contents/database
Content fetched successfully!


In [5]:
# extracting relevant information in a dict
data_file_sources = []
for data in content:
    # print(data['name'], data['download_url'])
    data_file_sources.append({
        'name': data['name'],
        'url': data['download_url']
    })

In [6]:
# let's take a look at the dict
print(len(data_file_sources))
print(data_file_sources[:1])

1000
[{'name': '1-59337-652-9.txt', 'url': 'https://raw.githubusercontent.com/alessandro1802/page-recommender-system/main/database/1-59337-652-9.txt'}]


In [7]:
# let's try to fetch the file content for the first file in the dataset
res = requests.get(data_file_sources[0]['url'])
content = res.text
text_data = '\n'.join(content.splitlines()[1:])
print(text_data[:100])

This page allows users to search multiple sources for a book given a 10- or 13-digit International S


In [8]:
%%time
# now let's fetch all the text data from all the files
raw_text_dataset = ''
for data in tqdm(data_file_sources):
    res = requests.get(data['url'])
    if res.status_code == requests.codes.ok:
        content = res.text
        text_data = '\n'.join(content.splitlines()[1:])
        raw_text_dataset += text_data + '\n\n'
    else:
        print("error fetching data from:", data['url'])

  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 23 s, sys: 16.4 s, total: 39.4 s
Wall time: 3min 42s


In [9]:
# length of dataset in characters
print("number of characters                 :", len(raw_text_dataset))
print("number of words using split          :", len(raw_text_dataset.split()))
print("number of words using regex w+       :", len(re.findall(r'\w+', raw_text_dataset)))
print("number of words without punctuations :", sum([i.strip(string.punctuation).isalpha() for i in raw_text_dataset.split()]))

# let's look at first 100 characters
print("first 100 characters:", raw_text_dataset[:100])

number of characters                 : 15795757
number of words using split          : 2438937
number of words using regex w+       : 2547081
number of words without punctuations : 2279952
first 100 characters: This page allows users to search multiple sources for a book given a 10- or 13-digit International S


In [10]:
# export and dump contents into a txt file
with open('raw-text-data.txt', 'w', encoding='utf-8') as f:
    f.write(raw_text_dataset)